In [33]:
#!pip install plyer
#from plyer import notification
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import geckodriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from datetime import datetime as date
import pandas as pd, numpy as np
import warnings as ws
ws.filterwarnings("ignore")
import re, os, time, glob
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pickle
from typing import List
import gc
# Create directory if it doesn't exist
directory = "Fetched Data\\Search results"
if not os.path.exists(directory):
    os.makedirs(directory)


def clean_str(string):
    string = string.replace('\xa0', " ").strip()
    string = string.replace('\n', "")
    string = string.replace('\t', "")
    return string.strip()

def extract_page_data(firefox):
    soup = BeautifulSoup(firefox.page_source, 'html.parser')
    table = soup.find('table', {'id':'ctl00_ContentPlaceHolder1_gvTramites'})
    headers = [th.text.strip() for th in table.find_all('th')]
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # Skip header row
        cells = row.find_all('td')
        row_data = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
        data.append(row_data)
    return data


# Initialize Firefox with options
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
geckodriver_autoinstaller.install()
firefox = webdriver.Firefox(options=options)

BASE_URL = 'http://helena.anmat.gob.ar/Boletin/'
firefox.get(BASE_URL)
WebDriverWait(firefox, timeout = 10)


    

Firefox version:  117.0.1
Available geckodriver version:  v0.34.0


<selenium.webdriver.support.wait.WebDriverWait (session="7ab6fbf1-1eaf-4fa7-ac3e-b5224161517c")>

In [36]:
data_dir = "Fetched Data"
page_pickle_dir = os.path.join(data_dir, "page_pickle")
def extract_page_data(firefox):
    soup = BeautifulSoup(firefox.page_source, 'html.parser')
    table = soup.find('table', {'id':'ctl00_ContentPlaceHolder1_gvTramites'})
    return table

In [37]:
page_extracts = dict()
view_state_cycle = 1

In [40]:


## Set manually
init = 1
count = 0
page_cut_at = 10


while count < page_cut_at:
    print(count)
    
    wait = WebDriverWait(firefox, 30)
    pagination_clikable = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'GridPager')))
    # Scroll to the bottom of the page
    firefox.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Save data to a CSV or JSON file
    #file_path = os.path.join(page_pickle_dir, f"page_{view_state_cycle}_{count}.xlsx") 
    page_extracts[str(view_state_cycle)+str(count+1)] =  extract_page_data(firefox)
    
    ## paginate
    pagination_div = firefox.find_element(By.CLASS_NAME,'GridPager').find_element(By.TAG_NAME,'tbody')
    firefox.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    next_page = pagination_div.find_elements(By.TAG_NAME, "td")[count+init]
    next_page.click()
    wait = WebDriverWait(firefox, 30)
    #pagination_clikable = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'GridPager')))
    count+= 1
    time.sleep(5)
    gc.collect()
    
    ## Switch loop
    if count == page_cut_at:
        print("Generate Next View State")
        count = 0
        init = 3
        gc.collect()
        view_state_cycle += 1
        if view_state_cycle > 65:
            break

0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4
5
6
7
8
9
Generate Next View State
0
1
2
3
4


IndexError: list index out of range

In [42]:
len(page_extracts.keys())

640

In [43]:
## Extract data from page extracts dict
collector = pd.DataFrame()
for page_num in page_extracts.keys():
    table = page_extracts.get(page_num)
    trs = table.find_all('tr')
    header = trs[0]
    for tr in trs[1:]:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.strip())
        row = pd.DataFrame(row)
        row = row.transpose()
        frame = [collector, row]
        collector = pd.concat(frame)
        
            
            

In [45]:
collector.shape

(33249, 15)

In [47]:
collector = collector.drop_duplicates()
collector.shape

(30541, 15)

In [57]:
collector = collector.dropna(axis=1)
collector.shape

(30541, 10)

In [58]:
collector.head(5)

,0,1,2,3,4,5,6,7,8,9
0,Registro Clase III-IV,25-01-2024,DROGUERIA MARTORANI S.A.,Apósito transparente I.V. reforzado con glucon...,IHT,"Oper film ®protect CHG 7 x 8,5 cm (Ref: 003400...",928-552,1-0047-3110-005060-23-9,,
0,Registro Clase III-IV,25-01-2024,TESIS SRL,Válvula cardíaca biológica,Meril,Aórticos:\nDDL19A; DDL21A; DDL23A; DDL25A; DDL...,867-44,1-0047-3110-005087-23-3,,
0,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de neuro-trombectomía,Neva Net,VN-6044-F3NR (NeVa IC-3 Net)\nVN-6035-F2NR (Ne...,954-232,1-0047-3110-005152-23-7,,
0,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de trombectomia coronaria,enVast,EV-4030-F2RR (enVast 4.0 x 30 mm)\nEV-4038-F3R...,954-233,1-0047-3110-005320-23-7,,
0,Registro Clase III-IV,25-01-2024,DIGIMED S.A.,Sistema de rayos X móvil,DRGEM,"JADE-32, JADE-40",1110-69,1-0047-3110-005598-23-9,,


In [59]:
headers = []
for th in header.find_all('th'):
    headers.append(th.text.strip())

In [60]:
collector.columns = headers

In [61]:
collector.head(5)

,Trámite,Fecha Fin,Razón Social,Nombre,Marca,Modelo/s,PM,Expediente,Dec/Cert,Disposición
0,Registro Clase III-IV,25-01-2024,DROGUERIA MARTORANI S.A.,Apósito transparente I.V. reforzado con glucon...,IHT,"Oper film ®protect CHG 7 x 8,5 cm (Ref: 003400...",928-552,1-0047-3110-005060-23-9,,
0,Registro Clase III-IV,25-01-2024,TESIS SRL,Válvula cardíaca biológica,Meril,Aórticos:\nDDL19A; DDL21A; DDL23A; DDL25A; DDL...,867-44,1-0047-3110-005087-23-3,,
0,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de neuro-trombectomía,Neva Net,VN-6044-F3NR (NeVa IC-3 Net)\nVN-6035-F2NR (Ne...,954-232,1-0047-3110-005152-23-7,,
0,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de trombectomia coronaria,enVast,EV-4030-F2RR (enVast 4.0 x 30 mm)\nEV-4038-F3R...,954-233,1-0047-3110-005320-23-7,,
0,Registro Clase III-IV,25-01-2024,DIGIMED S.A.,Sistema de rayos X móvil,DRGEM,"JADE-32, JADE-40",1110-69,1-0047-3110-005598-23-9,,


In [62]:
collector.to_excel("ANMAT_AR Full Extract_260124.xlsx")

In [22]:
## load previous chunks
from glob import glob as g
files = [pd.read_excel(file) for file in g(os.path.join("Fetched Data//chunks","*.*"))]
prev_data = pd.concat(files)

(11284, 15)

In [ ]:
anmat_AR = pd.concat([collector, prev_data])

In [7]:
pd.read_html(str(page_extracts['01']))[0]

,Trámite,Fecha Fin,Razón Social,Nombre,Marca,Modelo/s,PM,Expediente,Dec/Cert,Disposición,Unnamed: 10,Unnamed: 11
0,Registro Clase III-IV,25-01-2024,DROGUERIA MARTORANI S.A.,Apósito transparente I.V. reforzado con glucon...,IHT,"Oper film ®protect CHG 7 x 8,5 cm (Ref: 003400...",928-552,1-0047-3110-005060-23-9,NaN,NaN,NaN,NaN
1,Registro Clase III-IV,25-01-2024,TESIS SRL,Válvula cardíaca biológica,Meril,Aórticos: DDL19A; DDL21A; DDL23A; DDL25A; DDL2...,867-44,1-0047-3110-005087-23-3,NaN,NaN,NaN,NaN
2,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de neuro-trombectomía,Neva Net,VN-6044-F3NR (NeVa IC-3 Net) VN-6035-F2NR (NeV...,954-232,1-0047-3110-005152-23-7,NaN,NaN,NaN,NaN
3,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de trombectomia coronaria,enVast,EV-4030-F2RR (enVast 4.0 x 30 mm) EV-4038-F3RR...,954-233,1-0047-3110-005320-23-7,NaN,NaN,NaN,NaN
4,Registro Clase III-IV,25-01-2024,DIGIMED S.A.,Sistema de rayos X móvil,DRGEM,"JADE-32, JADE-40",1110-69,1-0047-3110-005598-23-9,NaN,NaN,NaN,NaN
5,Registro Clase III-IV,25-01-2024,MACOR INSUMOS HOSPITALARIOS SRL,Sistema de radiografía digital móvil Arco en C,"Fujifilm Medical System U.S.A., Inc",PERSONA C ref. RK0521S PERSONA C ref. RK0530S ...,872-128,1-0047-3110-005646-23-4,NaN,NaN,NaN,NaN
6,Registro Clase I-II,25-01-2024,BIO ANALITICA ARGENTINA S.A.,VIDEOSCOPIO GASTROINTESTINAl,Olympus,VIDEOSCOPIO GASTROINTESTINAL OLYMPUS GIF-1100 ...,1539-125,1-0047-3110-000191-24-1,NaN,NaN,NaN,NaN
7,Registro Clase I-II,25-01-2024,TECNOLOGIA LASER CORNEAL S.R.L.,BISTURÍES,AKTIVE,"C-12/AP - BISTURÍ CRESCENT 1,2 MM BISEL HACIA ...",2310-37,1-0047-3110-000327-24-2,NaN,NaN,NaN,NaN
8,Registro Clase I-II,25-01-2024,BM MEDICINE SRL,Prótesis de Miembro Inferior,METIZ,"Accesorios protésicos (tubos, adaptadores, enc...",2358-36,1-0047-3110-000343-24-7,NaN,NaN,NaN,NaN
9,Registro Clase I-II,25-01-2024,GASTROTEX SRL.,Masajeador con calor y vibración.,HYPERICE,"VENOM 2: incluye su accesorio para espalda, pi...",1701-13,1-0047-3110-000365-24-3,NaN,NaN,NaN,NaN


In [7]:
firefox.refresh()

In [5]:
### Pagination
view_state_nav = {}

pagination_div = firefox.find_element(By.CLASS_NAME,'GridPager').find_element(By.TAG_NAME,'tbody')
view_state = re.findall(r'\b\d+\b', pagination_div.text)
view_state

['41', '42', '43', '44', '45', '46', '47', '48', '49', '50']

In [6]:
view_state_pages = pagination_div.find_elements(By.TAG_NAME, "td")[:-1]

for page in view_state_pages[3:]:
    print(page.text)
    page.find_element(By.TAG_NAME,'a')
    
    

42
43
44
45
46
47
48
49
50
...


In [8]:
view_state_pages[12].text

'...'

In [8]:
firefox.refresh()

0
1
2
3
4
5
6
7
8


In [11]:
extract_page_data(firefox)

,Trámite,Fecha Fin,Razón Social,Nombre,Marca,Modelo/s,PM,Expediente,Dec/Cert,Disposición,Unnamed: 10,Unnamed: 11
0,Registro Clase I-II,25-01-2024,CALAMANTE HORACIO R. Y CALAMANTE N.G.,RESINA COMPUESTA FOTOPOLIMERIZABLE,ULTRADENT,"FORMA: A1E; A2E; A3E; A3,5E; A4E; B1E; B2E; WB...",640-274,1-0047-3110-000376-24-1,NaN,NaN,NaN,NaN
1,Registro Clase I-II,25-01-2024,MTG GROUP S.R.L.,Micro catéter,"INT / Shanghai Kindly Medical Instruments Co.,...","Microcatéter: MCS01-3, MCS01-3L, MCS02-3, MCS0...",1991-230,1-0047-3110-000391-24-2,NaN,NaN,NaN,NaN
2,Modificación de Registro Clase I-II,25-01-2024,MEGADENTAL S.A.,Poste para canal radicular,RTD,"Modelos: Cod: 46900003 KIT MACRO-LOCK OVAL, Co...",1139-99,1-0047-3110-000111-24-5,NaN,NaN,NaN,NaN
3,Modificación de Registro Clase I-II,25-01-2024,BECTON DICKINSON ARGENTINA SRL.,Catéter IV de seguridad,BD INSYTE™ AUTOGUARD™,381812 BD Insyte™ Autoguard™ Catéter intravasc...,634-38,1-0047-3110-000124-24-0,NaN,NaN,NaN,NaN
4,Registro Clase III-IV,25-01-2024,DROGUERIA MARTORANI S.A.,Apósito transparente I.V. reforzado con glucon...,IHT,"Oper film ®protect CHG 7 x 8,5 cm (Ref: 003400...",928-552,1-0047-3110-005060-23-9,NaN,NaN,NaN,NaN
5,Registro Clase III-IV,25-01-2024,TESIS SRL,Válvula cardíaca biológica,Meril,Aórticos: DDL19A; DDL21A; DDL23A; DDL25A; DDL2...,867-44,1-0047-3110-005087-23-3,NaN,NaN,NaN,NaN
6,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de neuro-trombectomía,Neva Net,VN-6044-F3NR (NeVa IC-3 Net) VN-6035-F2NR (NeV...,954-232,1-0047-3110-005152-23-7,NaN,NaN,NaN,NaN
7,Registro Clase III-IV,25-01-2024,UNIFARMA S.A.,Sistema de trombectomia coronaria,enVast,EV-4030-F2RR (enVast 4.0 x 30 mm) EV-4038-F3RR...,954-233,1-0047-3110-005320-23-7,NaN,NaN,NaN,NaN
8,Modificación de Registro Clase I-II,25-01-2024,BECTON DICKINSON ARGENTINA SRL.,Catéter intravascular de seguridad,BD Insyte™ Autoguard™ BC Pro,"24 GA 0,75 PULGADAS 22 GA 1,00 PULGADAS 20 GA ...",634-187,1-0047-3110-000126-24-8,NaN,NaN,NaN,NaN
9,Modificación de Registro Clase I-II,25-01-2024,C. D. G. SOCIEDAD ANONIMA,Compresa de gasa cortada,"Medispo, Medicon",Con / sin hebra radiopaca Medidas: varias medi...,2055-17,1-0047-3110-000200-24-2,NaN,NaN,NaN,NaN


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

2
3
4
5
6
7
8
9
10
...
>>


In [33]:
len(view_state_pages)

1